In [ ]:
import os
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

def analyze_single_csv(file_path):
    stock_name = os.path.basename(file_path).replace(".csv", "").upper()
    try:
        df = pd.read_csv(file_path)

        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'], dayfirst=True, errors='coerce')
            df = df.sort_values('Date')

        if 'Close' not in df.columns or df['Close'].isnull().all():
            return {"Stock": stock_name, "Error": "No 'Close' column or all values missing"}

        df['Daily Return'] = df['Close'].pct_change()

        mean_close = df['Close'].mean()
        median_close = df['Close'].median()
        std_close = df['Close'].std()
        mean_daily_return = df['Daily Return'].mean() * 100
        volatility = df['Daily Return'].std() * 100
        min_close = df['Close'].min()
        max_close = df['Close'].max()
        cumulative_return = ((df['Close'].iloc[-1] / df['Close'].iloc[0]) - 1) * 100 if len(df) > 1 else np.nan

        return {
            "Stock": stock_name,
            "Mean Close": round(mean_close, 3),
            "Median Close": round(median_close, 3),
            "Std Dev Close": round(std_close, 3),
            "Mean Daily Return (%)": round(mean_daily_return, 3),
            "Volatility (%)": round(volatility, 3),
            "Min Close": round(min_close, 3),
            "Max Close": round(max_close, 3),
            "Cumulative Return (%)": round(cumulative_return, 3)
        }

    except Exception as e:
        return {"Stock": stock_name, "Error": str(e)}

def quantitative_stock_analysis_parallel(folder_path, output_path="quantitative_summary.csv", max_workers=8):
    # Get all CSV files and sort alphabetically
    csv_files = sorted([os.path.join(folder_path, f) 
                        for f in os.listdir(folder_path) if f.lower().endswith(".csv")])

    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(analyze_single_csv, file) for file in csv_files]
        for future in as_completed(futures):
            results.append(future.result())

    summary_df = pd.DataFrame(results)

    # Sort the final DataFrame alphabetically by Stock
    summary_df = summary_df.sort_values('Stock').reset_index(drop=True)

    summary_df.to_csv(output_path, index=False)
    print(f"\n✅ Quantitative summary saved to: {output_path}")
    return summary_df

if _name_ == "_main_":
    folder_path = "/Users/nidhi/Desktop/GenAI_Hackathon/data/csv"
    output_file = "/Users/nidhi/Desktop/GenAI_Hackathon"

    summary_df = quantitative_stock_analysis_parallel(folder_path, output_file, max_workers=8)
    print(summary_df.head())